In [1]:
import optimizer as op
from predictionModels import ReductionDim, Analisis 
from sklearn.preprocessing import StandardScaler

2022-03-30 12:35:26.358928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-30 12:35:26.359105: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from dataclasses import dataclass

@dataclass
class ModelosItems:
    reduccion: ReductionDim
    analisis: Analisis
    val: float = 0.0

In [3]:
import pandas as pd
import numpy as np
import random
random.seed(1)
def balancedCut(df ,n ,frac = [0.5], target = 'Class',classes=[0,1]):
    print(f'{"="*50}\nbalancedCut\n{"="*50}')
    print(f'Realizando seleccion de la muestra la muestra')
    newDfs = []
    if sum(frac) > 1:
        print(f'Valor de la fraccion incorrecto: --> {sum(frac)} > 1')
        
    if len(frac) < len(classes):
        frac.append(1-sum(frac))
    
    for cl,fc in zip(classes, frac):
        ndf = df[df[target]==cl]
        ro = ndf.shape[0]
        print(f'Dataset de clase --> {cl}')
        if int(fc*n) > ro:
            newDfs.append(ndf)
            print(f'\tFilas: {ro}')
        else:
            newDfs.append(ndf.sample(n=int(fc*n)))
            print(f'\tFilas: {int(fc*n)}')
    
    return pd.concat(newDfs), newDfs[1].shape[0]/newDfs[0].shape[0]

In [4]:
cards = pd.read_csv("/home/vcalzada/Work/proyecto/modelos/creditcard.csv")
cards = cards.sample(frac=1)


cards.head()

df, frac = balancedCut(cards, 30000, frac=[0.9])
x = df.iloc[:,0:-1].to_numpy()

y = df.iloc[:,-1].to_numpy() 
scaler = StandardScaler()
x = scaler.fit_transform(x)

balancedCut
Realizando seleccion de la muestra la muestra
Dataset de clase --> 0
	Filas: 27000
Dataset de clase --> 1
	Filas: 492


In [5]:
reduccion={'pca':{'finalDim':[2,3,4]}, 
           'encoder':{'capas':[[32,16,8]], 'finalDim':2, 'batch_size':20, 'epochs':10 }}

In [6]:
modelos = {'isoForest':{'contamination':[0.1,0.01,-1]},
           'som':{'sigma':0.2, 'learning_rate':0.2, 'neighborhood_function':'gaussian', 'outliers_percentage':0.15, 'epochs':100}}

In [7]:
from predictionModels import ReductionDim, Analisis 
def optimVal(x, reduccion, modelo):
    reduc = reduccion.keys()
    redList = []
    mode = modelos.keys()
    modList = []
    for red in reduc:
        params = op.crossParam(reduccion[red])
        for par in params:
            aux = ReductionDim(red,x)
            aux.initParams(par)
            redList.append(aux)
    for mod in mode:
        params = op.crossParam(modelo[mod])
        for par in params:
            aux =  Analisis(mod)
            print(par)
            aux.initParams(par)
            modList.append(aux)
    return redList, modList
    
        

In [8]:
red, mod = optimVal(x, reduccion, modelos)

2022-03-30 12:35:39.544286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-30 12:35:39.544515: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-30 12:35:39.544589: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (L-1802005): /proc/driver/nvidia/version does not exist
2022-03-30 12:35:39.545799: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


0.1
0.01
-1
(0.2, 0.2, 'gaussian', 0.15, 100)


In [9]:
from itertools import product
k = list(product(red,mod))

In [10]:
obj = []
for i in k:
    obj.append(op.ModelosItems(*i))

In [11]:
for i in red:
    i.fit()

Epoch 1/10
1375/1375 [==============================] - 20s 14ms/step - loss: 0.9639 - val_loss: 0.9283
Epoch 2/10
1375/1375 [==============================] - 25s 18ms/step - loss: 0.9215 - val_loss: 0.9158
Epoch 3/10
1375/1375 [==============================] - 23s 16ms/step - loss: 0.9108 - val_loss: 0.9077
Epoch 4/10
1375/1375 [==============================] - 26s 19ms/step - loss: 0.9049 - val_loss: 0.9028
Epoch 5/10
1375/1375 [==============================] - 16s 11ms/step - loss: 0.9019 - val_loss: 0.8996
Epoch 6/10
1375/1375 [==============================] - 25s 18ms/step - loss: 0.9000 - val_loss: 0.8987
Epoch 7/10
1375/1375 [==============================] - 17s 12ms/step - loss: 0.8982 - val_loss: 0.8974
Epoch 8/10
1375/1375 [==============================] - 25s 18ms/step - loss: 0.8968 - val_loss: 0.8959
Epoch 9/10
1375/1375 [==============================] - 23s 17ms/step - loss: 0.8954 - val_loss: 0.8940
Epoch 10/10
1375/1375 [==============================] - 24s 17m

In [12]:
print(len(red), len(mod), len(obj))

4 4 16


In [13]:
op.ModelosItems.yVal(y)
for o in obj:
    print(o.y[[1,4,7,44]], o.metric)

[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision
[0 0 0 0] precision


In [14]:
frac

0.018222222222222223

In [15]:
obj[1].analisis.contamination

0.01

In [16]:
for o in obj:
    o.calMetricVal()

In [17]:
obj.sort(reverse=True)

In [22]:
print(obj[0])
print(obj[1])
print(obj[2])

___________________________________________________________________________
Reduccion --> pca
	Numero de parametros final --> 2
___________________________________________________________________________
___________________________________________________________________________
Analisis --> isoForest
	Contaminacion --> 0.1___________________________________________________________________________
___________________________________________________________________________
Reduccion --> pca
	Numero de parametros final --> 3
___________________________________________________________________________
___________________________________________________________________________
Analisis --> isoForest
	Contaminacion --> 0.1___________________________________________________________________________
___________________________________________________________________________
Reduccion --> pca
	Numero de parametros final --> 4
______________________________________________________________________

In [19]:
print(obj[5].reduccion.reductionN,obj[5].analisis.modeloN, obj[5].analisis.contamination)

AttributeError: 'Analisis' object has no attribute 'contamination'

In [ ]:
for o in obj:
    print(o.reduccion)